In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
import re
import time
import collections
import os
import itertools
from sklearn.cross_validation import train_test_split

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def add_start_end(string):
    string = string.split()
    strings = []
    for s in string:
        s = list(s)
        s[0] = '<%s'%(s[0])
        s[-1] = '%s>'%(s[-1])
        strings.extend(s)
    return strings

In [3]:
with open('lemmatization-en.txt','r') as fopen:
    texts = fopen.read().split('\n')
after, before = [], []
for i in texts[:10000]:
    splitted = i.encode('ascii', 'ignore').decode("utf-8").lower().split('\t')
    if len(splitted) < 2:
        continue
    
    after.append(add_start_end(splitted[0]))
    before.append(add_start_end(splitted[1]))
    
print(len(after),len(before))

10000 10000


In [4]:
def build_dataset(words, n_words, atleast=1):
    count = [['GO', 0], ['PAD', 1], ['EOS', 2], ['UNK', 3]]
    counter = collections.Counter(words).most_common(n_words)
    counter = [i for i in counter if i[1] >= atleast]
    count.extend(counter)
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [5]:
concat_from = list(itertools.chain(*before))
vocabulary_size_from = len(list(set(concat_from)))
data_from, count_from, dictionary_from, rev_dictionary_from = build_dataset(concat_from, vocabulary_size_from)
print('vocab from size: %d'%(vocabulary_size_from))
print('Most common words', count_from[4:10])
print('Sample data', data_from[:10], [rev_dictionary_from[i] for i in data_from[:10]])
print('filtered vocab size:',len(dictionary_from))
print("% of vocab used: {}%".format(round(len(dictionary_from)/vocabulary_size_from,4)*100))

vocab from size: 65
Most common words [('e', 9763), ('i', 7229), ('n', 6177), ('s>', 6095), ('o', 5769), ('a', 5633)]
Sample data [46, 5, 11, 14, 35, 47, 4, 6, 10, 39] ['<f', 'i', 'r', 's', 't>', '<t', 'e', 'n', 't', 'h>']
filtered vocab size: 69
% of vocab used: 106.15%


In [6]:
for i in range(len(after)):
    after[i].append('EOS')
after[0]

['<1>', 'EOS']

In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(before, after, test_size = 0.2)

In [8]:
GO = dictionary_from['GO']
PAD = dictionary_from['PAD']
EOS = dictionary_from['EOS']
UNK = dictionary_from['UNK']

In [9]:
def sent2idx(sent, vocab, UNK=UNK):
    tokens = sent
    oovs = []
    extend_tokens = []
    tokenized = []
    for token in tokens:
        if token not in vocab:
            tokenized.append(UNK)
            if token not in oovs:
                oovs.append(token)
            extend_tokens.append(len(vocab) + oovs.index(token))
        else:
            extend_tokens.append(vocab[token])
            tokenized.append(vocab[token])
    return tokenized, extend_tokens, oovs

def target2idx(sent, oovs, vocab,UNK=UNK):
    tokens = sent
    tokenized = []
    for token in tokens:
        if token not in vocab:
            if token not in oovs:
                tokenized.append(UNK)
            else:
                tokenized.append(len(vocab) + oovs.index(token))
        else:
            tokenized.append(vocab[token])
    return tokenized

In [10]:
import tensorflow as tf
import collections
from tensorflow.python.util import nest
from tensorflow.contrib.framework.python.framework import tensor_util
from tensorflow.python.layers.core import Dense

class CopyNetWrapperState(
    collections.namedtuple("CopyNetWrapperState", ("cell_state", "last_ids", "prob_c"))):
    def clone(self, **kwargs):
        def with_same_shape(old, new):
            """Check and set new tensor's shape."""
            if isinstance(old, tf.Tensor) and isinstance(new, tf.Tensor):
                return tensor_util.with_same_shape(old, new)
            return new

        return nest.map_structure(
            with_same_shape,
            self,
            super(CopyNetWrapperState, self)._replace(**kwargs))

class CopyNetWrapper(tf.contrib.rnn.RNNCell):
    '''
    A copynet RNN cell wrapper
    '''
    def __init__(self, cell, source_extend_tokens, max_oovs, encode_output, output_layer, vocab_size, name=None):
        '''
        Args:
            - cell: the decoder cell
            - source_extend_tokens: input tokens with oov word ids
            - max_oovs: max number of oov words in each batch
            - encode_output: the output of encoder cell
            - output_layer: the layer used to map decoder output to vocab distribution
            - vocab_size: this is target vocab size
        '''
        super(CopyNetWrapper, self).__init__(name=name)
        self.cell = cell
        self.source_extend_tokens = source_extend_tokens
        self.encode_output = encode_output
        self.max_oovs = max_oovs
        self.output_layer = output_layer
        self._output_size = vocab_size + max_oovs
        self.copy_layer = Dense(self.cell.output_size,activation=tf.tanh,use_bias=False,name="Copy_Weigth")

    def __call__(self, inputs, state):
        last_ids = state.last_ids
        prob_c = state.prob_c
        cell_state = state.cell_state
        
        '''
            - Selective read
                At first, my implement is based on the paper, which looks like belowing:
                    mask = tf.cast(tf.equal(tf.expand_dims(last_ids,1),self.source_extend_tokens), tf.float32)
                    pt = mask * prob_c
                    pt_sum = tf.reduce_sum(pt, axis=1)
                    pt = tf.where(tf.less(pt_sum, 1e-7), pt, pt / tf.expand_dims(pt_sum, axis=1))
                    selective_read = tf.einsum("ijk,ij->ik",self.encode_output, pt)
                It looks OK for me, but I got NAN loss after one training step. I tried tf.Print to debug, but cannot find 
                the problem. Then I tried tfdbg to detect the source of NAN and found it refer to the tf.where code piece, 
                which is 'pt / tf.expand_dims(pt_sum, axis=1)'. 
                I am not sure why this code will got NANs. Maybe pt and pt_sum are not of the same shape, due to the broadcasting,
                we will got NANs, and these NANs will be used to calculate the gradients, and cause NANs values.
                Take a close look at the paper, we will find that p(y_t,c|.) is same for all the same input tokens. And selective_read
                is sum of p_t * h_t, in which p_t is 1/K *(p(x_t,c|.)), K is sum of p(x_t,c|.). So p_t is equal to:
                p_t = 1 / (number of times x_t shows in input tokens)
        '''
        # get selective read
        # batch * input length
        mask = tf.cast(tf.equal(tf.expand_dims(last_ids,1),self.source_extend_tokens), tf.float32)
        mask_sum = tf.reduce_sum(mask, axis=1)
        mask = tf.where(tf.less(mask_sum, 1e-7), mask, mask / tf.expand_dims(mask_sum, axis=1))
        pt = mask * prob_c
        selective_read = tf.einsum("ijk,ij->ik",self.encode_output, pt)

        inputs = tf.concat([inputs, selective_read], axis=-1)
        outputs, cell_state = self.cell(inputs, cell_state)

        # this is generate mode
        vocab_dist = self.output_layer(outputs)

        # this is copy mode
        # batch * length * output size
        copy_score = self.copy_layer(self.encode_output)
        # batch * length
        copy_score = tf.einsum("ijk,ik->ij",copy_score,outputs)

        vocab_size = tf.shape(vocab_dist)[-1]
        extended_vsize = vocab_size + self.max_oovs 

        batch_size = tf.shape(vocab_dist)[0]
        extra_zeros = tf.zeros((batch_size, self.max_oovs))
        # batch * extend vocab size
        vocab_dists_extended = tf.concat(axis=-1, values=[vocab_dist, extra_zeros])

        # this part is same as that of point generator, but using einsum is much simpler.
        source_mask = tf.one_hot(self.source_extend_tokens,extended_vsize)
        attn_dists_projected = tf.einsum("ijn,ij->in", source_mask, copy_score)

        final_dist = vocab_dists_extended + attn_dists_projected

        # This is greeding search, need to test with beam search
        last_ids = tf.argmax(final_dist, axis=-1, output_type=tf.int32)

        # this is used to calculate p(y_t,c|.)
        # safe softmax
        final_dist_max = tf.expand_dims(tf.reduce_max(final_dist,axis=1), axis=1)
        final_dist_exp = tf.reduce_sum(tf.exp(final_dist - final_dist_max),axis=1)
        p_c = tf.exp(attn_dists_projected - final_dist_max) / tf.expand_dims(final_dist_exp, axis=1)
        p_c = tf.einsum("ijn,in->ij",source_mask,p_c)

        state = CopyNetWrapperState(cell_state=cell_state, last_ids=last_ids, prob_c=p_c)
        return final_dist, state

    @property
    def state_size(self):
        """size(s) of state(s) used by this cell.
            It can be represented by an Integer, a TensorShape or a tuple of Integers
            or TensorShapes.
        """
        return CopyNetWrapperState(cell_state=self.cell.state_size, last_ids=tf.TensorShape([]),
            prob_c = self.source_extend_tokens.shape[-1].value)

    @property
    def output_size(self):
        """Integer or TensorShape: size of outputs produced by this cell."""
        return self._output_size

    def zero_state(self, batch_size, dtype):
        with tf.name_scope(type(self).__name__ + "ZeroState", values=[batch_size]):
            cell_state = self.cell.zero_state(batch_size, dtype)
            last_ids = tf.zeros([batch_size], tf.int32) - 1
            prob_c = tf.zeros([batch_size, tf.shape(self.encode_output)[1]], tf.float32)
            return CopyNetWrapperState(cell_state=cell_state, last_ids=last_ids, prob_c=prob_c)

In [11]:
class Stemmer:
    def __init__(self, size_layer, num_layers, embedded_size, 
                 from_dict_size, learning_rate, max_oovs = 50):
        
        def lstm_cell(size = size_layer, reuse=False):
            return tf.nn.rnn_cell.GRUCell(size, reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.LuongAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell(reuse=reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        self.source_oov_words = tf.placeholder(tf.int32, shape=[])
        self.source_extend_tokens = tf.placeholder(tf.int32, shape=[None, None])
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        
        condition = tf.less(decoder_input, from_dict_size)
        self.decoder_input = self.Y
        self.decoder_input_length = self.Y_seq_len
        self.predict_count = tf.reduce_sum(self.decoder_input_length)
        encoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embeddings = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        for n in range(num_layers):
            (out_fw, out_bw), (state_fw, state_bw) = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = lstm_cell(size_layer // 2),
                cell_bw = lstm_cell(size_layer // 2),
                inputs = encoder_embedded,
                sequence_length = self.X_seq_len,
                dtype = tf.float32,
                scope = 'bidirectional_rnn_%d'%(n))
            encoder_embedded = tf.concat((out_fw, out_bw), 2)
        
        bi_state = tf.concat((state_fw, state_bw), -1)
        self.initial_state = tuple(bi_state for _ in range(num_layers))
        self.encoder_out = encoder_embedded
        decoder_cell = attention(self.encoder_out, self.X_seq_len)
        dense_layer = tf.layers.Dense(from_dict_size)
        
        initial_state = decoder_cell.zero_state(batch_size, tf.float32).clone(cell_state=self.initial_state)
        
        self.decode_cell = CopyNetWrapper(decoder_cell, self.source_extend_tokens, max_oovs, 
                                          self.encoder_out, dense_layer, from_dict_size)
        
        dense_layer = None
        
        self.initial_state = self.decode_cell.zero_state(batch_size,
                                                         tf.float32).clone(cell_state=initial_state)
        
        training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embeddings, decoder_input),
                sequence_length = self.decoder_input_length,
                time_major = False)
        training_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = self.decode_cell,
                helper = training_helper,
                initial_state = self.initial_state,
                output_layer = dense_layer)
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = training_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.Y_seq_len))
        
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = encoder_embeddings,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = self.decode_cell,
                helper = predicting_helper,
                initial_state = self.initial_state,
                output_layer = dense_layer)
        
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = True,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
        
        self.training_logits = training_decoder_output.rnn_output
        self.predicting_ids = predicting_decoder_output.sample_id
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [12]:
size_layer = 128
num_layers = 2
embedded_size = 64
learning_rate = 1e-3
batch_size = 32
epoch = 15

In [13]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Stemmer(size_layer, num_layers, embedded_size, len(dictionary_from), 
                learning_rate)
sess.run(tf.global_variables_initializer())

In [14]:
from keras.preprocessing.sequence import pad_sequences

def batching(X, Y):
    s_, es_, oovs_, target_ = [], [], [], []
    for x, y in zip(X, Y):
        s,es,oovs = sent2idx(x, dictionary_from)
        target = target2idx(y, oovs,dictionary_from)
        s_.append(s)
        es_.append(es)
        oovs_.append(oovs)
        target_.append(target)
    s_ = pad_sequences(s_,padding='post')
    es_ = pad_sequences(es_,padding='post')
    target_ = pad_sequences(target_,padding='post')
    maxlen = max([len(o) for o in oovs_])
    return s_, es_, target_, maxlen

Using TensorFlow backend.


In [15]:
from tqdm import tqdm
from sklearn.utils import shuffle
import time

for EPOCH in range(epoch):
    lasttime = time.time()
    total_loss, total_accuracy, total_loss_test, total_accuracy_test = 0, 0, 0, 0
    train_X, train_Y = shuffle(train_X, train_Y)
    test_X, test_Y = shuffle(test_X, test_Y)
    pbar = tqdm(range(0, len(train_X), batch_size), desc='train minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(train_X))
        batch_x, batch_es, batch_y, maxlen = batching(train_X[k: index],
                                                     train_Y[k: index])
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss += loss
        total_accuracy += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_X), batch_size), desc='test minibatch loop')
    for k in pbar:
        index = min(k+batch_size,len(test_X))
        batch_x, batch_es, batch_y, maxlen = batching(test_X[k: index],
                                                     test_Y[k: index])
        acc, loss = sess.run([model.accuracy, model.cost], 
                                      feed_dict={model.X:batch_x,
                                                 model.source_extend_tokens:batch_es,
                                                model.Y:batch_y,
                                                model.source_oov_words:maxlen})
        total_loss_test += loss
        total_accuracy_test += acc
        pbar.set_postfix(cost=loss, accuracy = acc)
        
    total_loss /= (len(train_X) / batch_size)
    total_accuracy /= (len(train_X) / batch_size)
    total_loss_test /= (len(test_X) / batch_size)
    total_accuracy_test /= (len(test_X) / batch_size)
        
    print('epoch: %d, avg loss: %f, avg accuracy: %f'%(EPOCH, total_loss, total_accuracy))
    print('epoch: %d, avg loss test: %f, avg accuracy test: %f'%(EPOCH, total_loss_test, total_accuracy_test))

train minibatch loop:   1%|          | 2/250 [00:00<00:18, 13.76it/s, accuracy=0.9, cost=0.401]

epoch: 0, avg loss: 1.276579, avg accuracy: 0.663112
epoch: 0, avg loss test: 0.442437, avg accuracy test: 0.898187


train minibatch loop:   1%|          | 2/250 [00:00<00:16, 14.95it/s, accuracy=0.971, cost=0.167]

epoch: 1, avg loss: 0.281214, avg accuracy: 0.929588
epoch: 1, avg loss test: 0.214072, avg accuracy test: 0.955397


train minibatch loop:   1%|          | 2/250 [00:00<00:17, 14.08it/s, accuracy=0.967, cost=0.172]

...

epoch: 12, avg loss: 0.020465, avg accuracy: 0.994062
epoch: 12, avg loss test: 0.059716, avg accuracy test: 0.996420


train minibatch loop:   1%|          | 2/250 [00:00<00:15, 15.71it/s, accuracy=0.996, cost=0.0193]

epoch: 13, avg loss: 0.020216, avg accuracy: 0.994379
epoch: 13, avg loss test: 0.055051, avg accuracy test: 0.996833


test minibatch loop: 100%|██████████| 63/63 [00:01<00:00, 34.40it/s, accuracy=1, cost=0.00429]    

epoch: 14, avg loss: 0.018538, avg accuracy: 0.995176
epoch: 14, avg loss test: 0.048555, avg accuracy test: 0.997163


In [22]:
predicted = sess.run(model.predicting_ids, 
                     feed_dict={model.X:batch_x,
                                model.source_extend_tokens:batch_es,
                                model.Y:batch_y,
                                model.source_oov_words:maxlen})

In [23]:
for i in range(len(batch_x)):
    print('row %d'%(i+1))
    print('BEFORE:',''.join([rev_dictionary_from[n] for n in batch_x[i] if n not in [0,1,2,3]]))
    print('REAL AFTER:',''.join([rev_dictionary_from[n] for n in batch_y[i] if n not in[0,1,2,3]]))
    print('PREDICTED AFTER:',''.join([rev_dictionary_from[n] for n in predicted[i] if n not in[0,1,2,3]]),'\n')

row 1
BEFORE: <assumed>
REAL AFTER: <assume>
PREDICTED AFTER: <assum>e>e>e>e>m>m>m>m>e>i 

row 2
BEFORE: <anaesthetizing>
REAL AFTER: <anaesthetize>
PREDICTED AFTER: <anaesthetie>e>e>e>e> 

row 3
BEFORE: <contacts>
REAL AFTER: <contact>
PREDICTED AFTER: <contaat>t>t>t>t>t>t>t>t> 

row 4
BEFORE: <arums>
REAL AFTER: <arum>
PREDICTED AFTER: <arm>m>m> 

row 5
BEFORE: <boxcars>
REAL AFTER: <boxcar>
PREDICTED AFTER: <boxcaa>e>e>e>i>ar>r>r>r> 

row 6
BEFORE: <batters>
REAL AFTER: <batter>
PREDICTED AFTER: <batteeeeeeeer>r>r> 

row 7
BEFORE: <compositions>
REAL AFTER: <composition>
PREDICTED AFTER: <compositiiie>e>e>t 

row 8
BEFORE: <acclimatizing>
REAL AFTER: <acclimatize>
PREDICTED AFTER: <acclimatie>e>e>e>e>e> 

row 9
BEFORE: <broadcasted>
REAL AFTER: <broadcast>
PREDICTED AFTER: <broadcastee>e>e>e>e> 

row 10
BEFORE: <building>
REAL AFTER: <build>
PREDICTED AFTER: <b<biild>e>e>e>e>iiiie> 

row 11
BEFORE: <botanies>
REAL AFTER: <botany>
PREDICTED AFTER: <botany>e>e>e>e>e>e>e>e>e> 

row 12
